In [1]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
global_x = 0
global_y = 0

def mouse_callback(event, _x, _y, flags, param):
    global global_x
    global global_y
    if event == cv2.EVENT_MOUSEMOVE:
        global_x, global_y = _x, _y
        
cv2.namedWindow('webcam')      
cv2.setMouseCallback('webcam', mouse_callback)

while True:
    ret, frame = cap.read()
    cur_frame = cv2.resize(frame,None,fx=1,fy=1,interpolation=cv2.INTER_AREA)
    height, width, channels = cur_frame.shape
    grey_frame = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
    grey_frame = np.float32(grey_frame)
    
    harris = cv2.cornerHarris(grey_frame, blockSize=5, ksize=5, k=0.04)
    harris = cv2.dilate(harris, None)
    cur_frame[harris > 0.0009*harris.max()] = [255,0,0]
    '''max, max_i, max_j = 0, 0, 0
    for row in range (0, height, 1):
        for col in range (0, width, 1):
            for i in range (row, row+10):
                for j in range (col, col+10):
                    if (i>=height-1 and j>=width-1):
                        break
                    elif (max < harris[i, j]):
                        max = harris[i, j]
                        max_i = i
                        max_j = j
            cur_frame[max_i, max_j] = [0, 0, 255]'''
    
    # Draw
    show_str = 'x:{} y:{} '.format(global_x, global_y)
    harris_val_str = 'resp. val:{}'.format(harris[global_y, global_x])
    cv2.rectangle(cur_frame, (10, 0), (224, 30), (255, 255, 255), -1)
    cv2.putText(cur_frame, show_str, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
    cv2.putText(cur_frame, harris_val_str, (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
    # Draw End

    cv2.imshow('webcam', cur_frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
cap.release()
cv2.destroyAllWindows()